<a href="https://colab.research.google.com/github/meta-llama/llama-recipes/blob/main/recipes/use_cases/agents/langchain/tool-calling-agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -U langchain_groq langchain langchain_community sentence-transformers tavily-python tiktoken langchainhub chromadb

# Tool calling agent with Llama 3

[Tool calling](https://python.langchain.com/docs/modules/agents/agent_types/tool_calling/) allows an LLM to detect when one or more tools should be called.

It will then respond with the inputs that should be passed to those tools. 

LangChain has a general agent that works with tool-calling LLMs.

### Tools 

Let's define a few tools.

`Retriever`

In [ ]:
import os

os.environ['TAVILY_API_KEY'] = 'YOUR_TAVILY_API_KEY'
os.environ['GROQ_API_KEY'] = 'YOUR_GROQ_API_KEY'

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = Chroma.from_documents(documents, HuggingFaceEmbeddings())
retriever = vector.as_retriever()

from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

`Web search`

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()

`Custom`

In [ ]:
from langchain.agents import tool

@tool
def magic_function(input: int) -> int:
    """Applies a magic function to an input."""
    return input + 2

In [ ]:
tools = [magic_function, search, retriever_tool] 

### LLM

Here, we need a Llama 3 model that supports tool use.

This can be accomplished via prompt engineering (e.g., see [here](https://replicate.com/hamelsmu/llama-3-70b-instruct-awq-with-tools)) or fine-tuning (e.g., see [here](https://huggingface.co/mzbac/llama-3-8B-Instruct-function-calling) and [here](https://huggingface.co/mzbac/llama-3-8B-Instruct-function-calling)).

We can review LLMs that support tool calling [here](https://python.langchain.com/docs/integrations/chat/) and Groq is included.

[Here](https://github.com/groq/groq-api-cookbook/blob/main/llama3-stock-market-function-calling/llama3-stock-market-function-calling.ipynb) is a notebook by Groq on function calling with Llama 3 and LangChain.

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(temperature=0, model="llama3-70b-8192")

### Agent

We use LangChain [tool calling agent](https://python.langchain.com/docs/modules/agents/agent_types/tool_calling/). 

In [ ]:
# Prompt 
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)
prompt.pretty_print()

In [ ]:
### Run
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "what is the value of magic_function(3)?"})

Trace: 

https://smith.langchain.com/public/adf06494-94d6-4e93-98f3-60e65d2f2c19/r

In [ ]:
agent_executor.invoke({"input": "whats the weather in sf?"})

Trace: https://smith.langchain.com/public/64a62781-7e3c-4acf-ae72-ce49ccb82960/r

In [ ]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})

This last question failed to run. 

Trace: https://smith.langchain.com/public/960a40e9-24f1-42a0-859d-2e0a30018d1c/r

We can see that the agent correctly decides to query the vectorstore for a question about LangSmith. But it then inexplicably attempts web search. And it appears to get stuck in a loop of calling various tools before crashing.

Of course, this is using a non-fine-tuned (only prompting) version of llama3 for tool-use. But, it illustates the reliability challenge with using Agent Executor. It is sensitive to the LLM's capacity for tool-use! 

In the next notebook, we will show an alternative way to implement this agent using LangGraph.